<a href="https://colab.research.google.com/github/modoukebe/depot2/blob/main/appeemapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geemap

In [9]:
ee.Initialize()

In [ ]:
Map = geemap.Map()
Map